In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from models import *
import numpy as np
import time
import cv2
from PIL import Image
import torch.utils.data as Data
import re
from torch.utils.data import Dataset

import torchnvjpeg
from multiprocessing.pool import ThreadPool

In [11]:
num_images = 50000
hidden_dimension  = 15
weights = []
batch_size = 128
start_epoch = 0
lr = 0.01
test_nSamples = 10000
init_width = 32
init_height = 32
batch_size_test = 100

In [12]:
imgDir = './cifar_10_images/train_cifar10'
test_imgDir = './cifar_10_images/test_cifar10'

In [13]:
batch_size = 128
max_cpu_threads = 8
device_id = 0
max_image_size = 32 * 32 *3 
#image_path = "/export/hdd/scratch/hchen799/JPEG_GPU_decode/torchnvjpeg/images/cat.jpg"
#data = open(image_path, 'rb').read()
#data_list = [data for _ in range(batch_size)]

#decoder_list = [torchnvjpeg.Decoder() for _ in range(batch_size)]

In [14]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

In [15]:
def load_image_path(imgDir):

    all_training_files=os.walk(imgDir)
    train_files=[]
    train_imageNames=[]
    train_nSamples=0
    for path,direction,filelist in all_training_files:
        files = [file for file in filelist if os.path.isfile(os.path.join(path, file))]
        imageNames = [file.split('.')[0] for file in files if is_image_file(file)]
        files = [os.path.join(path, file) for file in files if is_image_file(file)]
        train_files.append(files)
        train_imageNames.append(imageNames)
        train_nSamples=train_nSamples+len(files)
    train_files=sum(train_files,[])
    train_imageNames=sum(train_imageNames,[])
    #print(train_imageNames)
    train_imageNames.sort(key = lambda i:int(re.match(r'(\d+)',i).group()))
    #train_imageNames.sort(key = lambda x: int(x[:-4]))
    train_image_path = []
    for i in range (len(train_imageNames)):
        string = imgDir + '/' + train_imageNames[i] + '.jpg'
        train_image_path.append(string)
    return train_image_path

In [16]:
def load_image_path_test(imgDir):

    all_training_files=os.walk(imgDir)
    train_files=[]
    train_imageNames=[]
    train_nSamples=0
    for path,direction,filelist in all_training_files:
        files = [file for file in filelist if os.path.isfile(os.path.join(path, file))]
        imageNames = [file.split('.')[0] for file in files if is_image_file(file)]
        files = [os.path.join(path, file) for file in files if is_image_file(file)]
        train_files.append(files)
        train_imageNames.append(imageNames)
        train_nSamples=train_nSamples+len(files)
    train_files=sum(train_files,[])
    train_imageNames=sum(train_imageNames,[])
    #print(train_imageNames)
    train_imageNames.sort(key = lambda i:int(re.match(r'(\d+)',i).group()))
    #train_imageNames.sort(key = lambda x: int(x[:-4]))
    train_image_path = []
    for i in range (len(train_imageNames)):
        string = imgDir + '/' + train_imageNames[i] + '.jpg'
        train_image_path.append(string)
    return train_image_path

In [17]:
train_image_path = load_image_path(imgDir)
print(train_image_path[0])

./cifar_10_images/train_cifar10/0.jpg


In [18]:
index_list = torch.randperm(50000)

In [19]:
print(index_list)

tensor([21043,  9611,  5466,  ..., 36841, 41202, 15014])


In [21]:
# iteration = int(50000/batch_size)
# iteration = 1
# decoder = torchnvjpeg.Decoder(0, 0, True, device_id, batch_size, max_cpu_threads, max_image_size, torch.cuda.current_stream(device_id))
# start_decoding_time = time.time()
# for i in range (iteration):
#     jpeg_cpu_data_list = []
#     for j in range (batch_size):
#         index = index_list[i * batch_size + j]
#         index = 0
#         jpeg_cpu = open(train_image_path[index], 'rb').read()
#         jpeg_cpu_data_list.append(jpeg_cpu)
#     image_tensor_list = decoder.batch_decode(jpeg_cpu_data_list)
# end_encoding_time = time.time()
# print("the JPEG decoding time on GPU is",end_encoding_time - start_decoding_time)
        

In [23]:
#print(image_tensor_list[0])

In [24]:
transform_train = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize((0.48947954, 0.48043759, 0.4465), (0.24513403, 0.24289503, 0.25973428)),
])

In [25]:
imgpath = train_image_path[0]
img = Image.open(imgpath).convert('RGB')
print(img)

<PIL.Image.Image image mode=RGB size=32x32 at 0x7F8BBD4968B0>


In [26]:
image_array = np.asarray(img)
print(image_array.shape)

(32, 32, 3)


In [27]:
img_tensor = transform_train(img)
print(img_tensor.shape)
print(img_tensor * 255)

torch.Size([3, 32, 32])
tensor([[[ 58.,  44.,  55.,  ..., 158., 151., 149.],
         [ 19.,   3.,  20.,  ..., 120., 116., 124.],
         [ 25.,  17.,  50.,  ..., 122., 124., 112.],
         ...,
         [213., 195., 198.,  ..., 160.,  55.,  58.],
         [180., 164., 179.,  ..., 179.,  88.,  78.],
         [181., 166., 177.,  ..., 219., 147., 115.]],

        [[ 61.,  44.,  47.,  ..., 134., 127., 123.],
         [ 21.,   0.,   8.,  ...,  87.,  83.,  89.],
         [ 23.,   7.,  28.,  ...,  82.,  84.,  71.],
         ...,
         [168., 151., 159.,  ..., 134.,  30.,  30.],
         [139., 126., 144.,  ..., 148.,  63.,  56.],
         [145., 132., 145.,  ..., 182., 121.,  97.]],

        [[ 66.,  42.,  36.,  ..., 106.,  99.,  98.],
         [ 18.,   0.,   0.,  ...,  54.,  50.,  59.],
         [ 11.,   0.,  14.,  ...,  46.,  48.,  39.],
         ...,
         [ 83.,  52.,  42.,  ...,  85.,   0.,  26.],
         [ 85.,  55.,  50.,  ..., 104.,  33.,  45.],
         [111.,  86.,  72.,  

In [28]:
print(img)

<PIL.Image.Image image mode=RGB size=32x32 at 0x7F8BBD4968B0>
